<a href="https://colab.research.google.com/github/HI-IDN/IDeLM-APAP-learning-fairness-models/blob/main/Anesthesiologist's_peel_assignment_problem_(APAP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code needs a cleanup ...

Dev notes: If the data entered is not clean and verified one may end up with no feasible solution found, we need to check if the data can generate a feasible solution before continuing.

### Anesthesiologist's peel assignment problem (APAP)



Anesthesiologists have different roles and shifts depending on the type of surgery, the level of care, and the availability of staff. Here is a brief description and summary of some of the common types of shifts and roles that anesthesiologists may have at a hospital:

- Cardiac: This is an anesthesiologist who specializes in providing anesthesia for patients undergoing cardiac surgery, such as coronary bypass, valve replacement, or heart transplant. They have advanced training and certification in transesophageal echocardiography (TEE), which is a technique that uses ultrasound to monitor the heart function during surgery. They also manage the hemodynamic and coagulation status of the patient, and may be involved in postoperative care in the cardiac intensive care unit (CVICU).
- Charge: This is an anesthesiologist who is responsible for overseeing the daily operations of the anesthesia department, such as assigning cases, coordinating staff, managing emergencies, and ensuring quality and safety standards. They may also provide clinical care for some cases, depending on the workload and staffing.
- Post call: This is an anesthesiologist who has completed a night or weekend call shift, which usually involves providing anesthesia for urgent or emergent cases that require immediate attention. They may have a reduced workload or be off duty the following day, depending on the hospital policy and their personal preference.
- Post late: This is an anesthesiologist who has worked past their scheduled shift time, usually due to a prolonged or complicated case. They may have a reduced workload or be off duty the following day, depending on the hospital policy and their personal preference.
- Post CVCC: This is an anesthesiologist who has provided anesthesia for a cardiac case and has followed the patient to the CVICU for postoperative care. They may have a reduced workload or be off duty the following day, depending on the hospital policy and their personal preference.
- Pre call: This is an anesthesiologist who is scheduled to work a night or weekend call shift. They may have a reduced workload or be off duty during the day, depending on the hospital policy and their personal preference.
- Late: This is an anesthesiologist who is scheduled to work until a later time than the regular shift, usually to cover for cases that start late in the day or extend into the evening. They may have a reduced workload or be off duty the following day, depending on the hospital policy and their personal preference.
- Call: This is an anesthesiologist who is on duty to provide anesthesia for urgent or emergent cases that require immediate attention. They may work up to 12-hour shifts and be on call 24/7. They may have a reduced workload or be off duty the following day, depending on the hospital policy and their personal preference.

These are some of the common types of shifts and roles that anesthesiologists may have at a hospital, but they may vary depending on the specific setting, practice model, and individual preference.

*We need to describe the specifics of the **equity** problem addressed in the model.*



In [ ]:
# select the Sheet and Google document ID you want to work with (make sure everyone has Editor access)
sheet_name = 'Test'
document_id = '1eUW2qvpyXIL7ZXKkGsWdUo0A6kZ_j-3faJ5TCPd8pXs'

In [ ]:
# load packages required
!python -m pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.gurobi.com


In [ ]:
# Authorize access to google doc
gc = gspread.authorize(creds)
# Open the spreadsheet by its key
sh = gc.open_by_key(document_id)
# Get the first worksheet of the spreadsheet
worksheet = sh.worksheet(sheet_name)
df = worksheet.get('A1:F40')
df = pd.DataFrame(worksheet.get('A2:F40'), columns=worksheet.get('A1:F1')).fillna('')

In [ ]:
# Sets and objects
Anst = ['AY', 'BK', 'CA', 'CC', 'CM', 'DD', 'DN', 'ES', 'FM', 'JJ', 'JM', 'JT', 'KC', 'KK', 'MA', 'MC', 'MI', 'RR', 'SK', 'SL', 'TI', 'TT']
Diac = ['BK', 'CA', 'CC', 'DD', 'ES', 'FM', 'JJ', 'JM', 'JT', 'KC', 'RR', 'SK', 'SL']
Chrg = ['CA', 'CC', 'DN', 'JJ', 'JM', 'KC', 'MC', 'TI']
Wday = ['MON', 'TUE', 'WED', 'THU', 'FRI']
Peel = list(range(4,15))

In [ ]:
Whine = {key: [] for key in Wday}
for i in range(1,6):
  idx = list(range(0,7))+list(range(18,20))+list(range(20,22))+list(range(22,39))
  col = df.iloc[idx,i]
  Assg = (col[col!=''].tolist())
  Whine[Wday[i-1]] = list(set(Anst) - set(Assg))
  Whine[Wday[i-1]].sort()
Whine

{'MON': ['AY', 'CC', 'CM', 'DN', 'KC', 'MC', 'SK', 'TT'],
 'TUE': ['AY', 'CA', 'CC', 'DN', 'JT', 'MC', 'TT'],
 'WED': ['BK', 'CA', 'CC', 'DN', 'MC', 'RR', 'TT'],
 'THU': ['BK', 'CC', 'CM', 'DN', 'FM', 'JT', 'TT'],
 'FRI': ['CA', 'CC', 'CM', 'DN', 'ES', 'FM', 'JT', 'KC']}

In [ ]:
# Now lets compute the current points count pre-assigned numbers
Pnts0 = {key: 0 for key in Anst}
Assg0 = {key: 0 for key in Anst} # if pre-assigned too often then fixed >= 4 or 3 ?
Prep0 = {key: 0 for key in Wday} # the persons assigned in slots 2,3,...,6
Rank0 = {key: 9 for key in Wday}
idx = [2,3,4,5,6]
for j in range(1,6):
  rank = 1
  for k in range(len(idx)):
    i = idx[k]
    if (df.iloc[i,j] != ''):
      Pnts0[df.iloc[i,j]] = Pnts0[df.iloc[i,j]] + rank
      Assg0[df.iloc[i,j]] = Assg0[df.iloc[i,j]] + 1
      rank = rank + 1
  Rank0[Wday[j-1]] = rank
  Prep0[Wday[j-1]] = np.sum(df.iloc[:7,j] != '') - 4
idx = [18,19]
for j in range(1,6):
  rank = Rank0[Wday[j-1]] + len(Whine[Wday[j-1]])
  for k in range(len(idx)):
    i = idx[k]
    if (df.iloc[i,j] != ''):
      Pnts0[df.iloc[i,j]] = Pnts0[df.iloc[i,j]] + rank
      rank = rank + 1
idx = [20,21]
for j in range(1,6):
  AdminPts = 8
  for k in range(len(idx)):
    i = idx[k]
    if (df.iloc[i,j] != ''):
      Pnts0[df.iloc[i,j]] = Pnts0[df.iloc[i,j]] + AdminPts

In [ ]:
# Lets see if we have a missing cardio of charge per day:
mCharge = {key: True for key in Wday}
mCardio = {key: True for key in Wday}
for i in range(1,6):
  d = Wday[i-1]
  # check for charge
  for j in [18,19]:
    if df.iloc[j,i] in Chrg:
       mCharge[d] = False
    if df.iloc[j,i] in Diac:
       mCardio[d] = False

In [ ]:
onCall = {Wday[i-1]: df.iloc[19,i] for i in range(1,6)}
onLate = {Wday[i-1]: df.iloc[18,i] for i in range(1,6)}

In [ ]:
# Scan the Whine zone for pre-assigned (needed to block peel positions for special requests)
SpPeel = {}
for i in range(1,6):
  for j in range(7,18):
    if df.iloc[j,i] != '':
      a = df.iloc[j,i]
      d = Wday[i-1]
      p = j-3
      SpPeel[(a,d,p)] = True

In [ ]:
# Initialize model
m = gp.Model("Anesthesiologist peel assignment problem (APAP)")

# Filtered sets for compactness
AWP = [(a,d,p) for d in Wday for a in Whine[d] for p in Peel[:(1+len(Whine[d]))]]
AdW = [(a,d) for d in Wday for a in Whine[d]+[onCall[d]]+[onLate[d]] if a in Diac]
AcW = [(a,d) for d in Wday for a in Whine[d]+[onCall[d]]+[onLate[d]] if a in Chrg]

# Assigned position
x = m.addVars(AWP, vtype = "B")
# Compute sum of points
y = m.addVars(Anst, vtype = "C")
# Maximum points per any individual
z1 = m.addVar()
z2 = m.addVar()
z3 = m.addVar()
z4 = m.addVar()
# Whos is the Cardio?
h = m.addVars(AdW, vtype = "B")
# Whos is the Charge?
c = m.addVars(AcW, vtype = "B")
zcha = m.addVar() # max combined per a in Anst
zch = m.addVars(Anst)

# Those that have not been assigned to a slot yet must be assigned:
m.addConstrs(gp.quicksum(x[a,d,p] for p in Peel[:(1+len(Whine[d]))]) == 1 for d in Wday for a in Whine[d])

# Can only assign one per peel per wday
m.addConstrs(gp.quicksum(x[a,d,p] for a in Whine[d]) <= 1 for d in Wday for p in Peel[:(1+len(Whine[d]))])

# The approximate amount of points assigned to each Anesthetist
m.addConstrs(gp.quicksum(x[a,d,p]*(p+Prep0[d]) for d in Wday for p in Peel if (a,d,p) in AWP) + Pnts0[a] == y[a] for a in Anst)

# The maximum number (two layered press!)
m.addConstrs(y[a] <= z1 for a in Anst if Assg0[a] < 4)
m.addConstrs(y[a] <= z2 for a in Anst)
m.addConstrs(y[a] >= z3 for a in Anst if Assg0[a] < 4)
m.addConstrs(y[a] >= z4 for a in Anst)

# now for the special conditions:
# If we have a Cardio or Charge missing from any given day, then we should force them to be late out
m.addConstrs(gp.quicksum(x[a,d,p] for a in Diac for p in Peel[(len(Whine[d])-1):(1+len(Whine[d]))] if (a,d) in AdW) >= 1 for d in Wday if mCardio[d])
m.addConstrs(gp.quicksum(x[a,d,p] for a in Chrg for p in Peel[(len(Whine[d])-1):(1+len(Whine[d]))] if (a,d) in AcW) >= 1 for d in Wday if mCharge[d])

# Tricky condition, minimizing the number of Cardio and Charge over the week, it works!
m.addConstrs(gp.quicksum(c[a,d] for a in Whine[d]+[onCall[d]]+[onLate[d]] if a in Chrg) == 1 for d in Wday)
m.addConstrs(gp.quicksum(h[a,d] for a in Whine[d]+[onCall[d]]+[onLate[d]] if a in Diac) == 1 for d in Wday)
# The same person cannot take both roles:
m.addConstrs(gp.quicksum(h[a,d]+c[a,d] for a in Chrg if (a,d) in AdW and (a,d) in AcW) <= 1 for d in Wday)
# Now if we have decided the role then they must either be late or on call
m.addConstrs(gp.quicksum(x[a,d,p] for p in Peel[(len(Whine[d])-1):(1+len(Whine[d]))]) >= h[a,d] for (a,d) in AdW if (a != onCall[d]) and (a != onLate[d]))
m.addConstrs(gp.quicksum(x[a,d,p] for p in Peel[(len(Whine[d])-1):(1+len(Whine[d]))]) >= c[a,d] for (a,d) in AcW if (a != onCall[d]) and (a != onLate[d]))
# now calculate the maximum number on Cardio or Charge
m.addConstrs(gp.quicksum(h[a,d] for d in Wday if (a,d) in AdW)+gp.quicksum(c[a,d] for d in Wday if (a,d) in AcW) <= zch[a] for a in Chrg if a in Diac)
m.addConstrs(gp.quicksum(c[a,d] for d in Wday if (a,d) in AcW) <= zch[a] for a in Chrg)
m.addConstrs(gp.quicksum(h[a,d] for d in Wday if (a,d) in AdW) <= zch[a] for a in Diac)
m.addConstrs(zch[a] <= zcha for a in Diac)

# Only once in the end peel
m.addConstrs(gp.quicksum(x[a,d,p] for d in Wday for p in Peel[len(Whine[d])-1:len(Whine[d])] if (a,d,p) in AWP) <= 1 for a in Anst)

# Special requests, fixed for Whine[d]
m.addConstrs(x[a,d,p] == 1 for (a,d,p) in SpPeel.keys() if (a,d,p) in AWP)

# minimize the maximum (worst case)
m.setObjective(gp.quicksum(y[a] for a in Anst) + gp.quicksum(zch[a] for a in Anst) + 10000*z1 + 10000*z2 - 1*z3 - 1*z4 + 100*zcha, GRB.MINIMIZE)
#m.setObjective(gp.quicksum(y[a] for a in Anst) + 1000*z2 + gp.quicksum(zch[a] for a in Anst), GRB.MINIMIZE)

# Optimize model
m.optimize()

# Display result:
df_soln = df.copy()
for i in range(1,6):
  d = Wday[i-1]
  for a in Whine[d]:
    for p in Peel[:(1+len(Whine[d]))]:
      if x[a,d,p].X > 0.5:
        #print(a,d,p)
        df_soln.iloc[3+p,i] = a
        if (a in Diac):
          if (h[a,d].X > 0.5):
            df_soln.iloc[3+p,i] = a+'*'
        if (a in Chrg):
          if (c[a,d].X > 0.5):
            df_soln.iloc[3+p,i] = a+'+'
  for j in [18,19]: # could have used here onCall on Late
    tmp = df.iloc[j,i]
    if (tmp in Chrg):
      if (c[tmp,d].X > 0.5):
        df_soln.iloc[j,i] = df_soln.iloc[j,i] + '+'
    if (tmp in Diac):
      if (h[tmp,d].X > 0.5):
        df_soln.iloc[j,i] = df_soln.iloc[j,i] + '*'

# display the points per person:
print("Points per person:")
ppp = {}
for a in Anst:
  if (int(y[a].X) > 0):
    print(int(y[a].X)+0*Pnts0[a],":", a)
    ppp[a] = int(y[a].X+0*Pnts0[a])

for (a,d) in AcW:
  if (int(c[a,d].X) > 0):
    print(a," is on charge on ", d)
for (a,d) in AdW:
  if (int(h[a,d].X) > 0):
    print(a," is on cardio on ", d)
print("max number of cardio+charge=", zcha.X)
df_soln

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 306 rows, 408 columns and 1495 nonzeros
Model fingerprint: 0x327b24ab
Variable types: 49 continuous, 359 integer (359 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 1100683.0000
Presolve removed 122 rows and 40 columns
Presolve time: 0.01s
Presolved: 184 rows, 368 columns, 1199 nonzeros
Variable types: 0 continuous, 368 integer (353 binary)
Found heuristic solution: objective 1100682.0000

Root relaxation: objective 7.005351e+05, 442 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

,Weekday,MON,TUE,WED,THU,FRI
0,Date,,,,,
1,Post Gill,TI,MI,JJ,TI,MI
2,Post CVCC,,,,,
3,Post Call,CA,BK,FM,KC,AY
4,Post Weekend/Holiday,ES,,,,
5,Post Late,JT,RR,CM,ES,RR
6,Pre Call,FM,KC,AY,MC,TT
7,,DN,MC,BK,BK,CC
8,,MC,CA,CC,TT,CM
9,,CC,JT,DN,JT,KC


In [ ]:
Peel

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
# Write the solution to the document
worksheet.update('G9:K17',df_soln.iloc[7:16,1:6].to_numpy().tolist())
worksheet.update('G20:K21',df_soln.iloc[18:20,1:6].to_numpy().tolist())
worksheet.update('L2:M'+str(len(ppp)+1),[[a,ppp[a]] for a in ppp.keys()])
df_soln

,Weekday,MON,TUE,WED,THU,FRI
0,Date,,,,,
1,Post Gill,TI,MI,JJ,TI,MI
2,Post CVCC,,,,,
3,Post Call,CA,BK,FM,KC,AY
4,Post Weekend/Holiday,ES,,,,
5,Post Late,JT,RR,CM,ES,RR
6,Pre Call,FM,KC,AY,MC,TT
7,,DN,MC,BK,BK,CC
8,,MC,CA,CC,TT,CM
9,,CC,JT,DN,JT,KC
